# Q-learning Environment

In [554]:
import numpy as np
import random as rand

# Giocatore

In [555]:
class Giocatore():

    #costruttore
    def __init__(self):
        self.mano = [] #carte in mano
        self.turno = False #ha il turno
        self.punteggio = 0

    #giocatore prende la prima carta del mazzo
    def pesca(self, mazzo):
        carta_pescata = mazzo.pesca()
        self.mano.append(carta_pescata) #aggiunge la carta pescata in mano

    #giocatore fa mossa random
    def mossaRandom(self, partita):
        rand.shuffle(self.mano) #mischia le carte in mano
        if self.mano != []:
            if(partita.first_throw == None): #controlla se è il primo a tirare
                partita.setThrow(self, self.mano.pop(0), True) #set della giocata fatta per primo
            else:
                partita.setThrow(self, self.mano.pop(0), False) #set della giocata fatta per secondo
            
    #funzione che setta il punteggio
    def setPunteggio(self, punti):
        self.punteggio = self.punteggio + punti

# Carta

In [556]:
class Carta():
    
    semi = ["denara", "bastoni", "spade", "coppe"] #tutti i semi del gioco

    #costruttore
    def __init__(self, seme, valore):
        self.seme = seme
        self.valore = valore #asso, due, tre, quattro, cinque, sei, sette, fante, cavallo, re
        self.definisciPunti() #definisce i punti della carta

    #funzione che definisce i punti della carta
    def definisciPunti(self):
        if self.valore == 1: #asso
            self.punti = 11
        elif self.valore == 3: #tre
            self.punti = 10
        elif self.valore > 7: #fante, cavallo, re
            self.punti = self.valore - 6
        else:
            self.punti = 0 #tutte le altre

# Mazzo

In [557]:
class Mazzo():
    
    #costruttore
    def __init__(self):
        self.carte = self.creaCarte() #crea tutte le carte
        rand.shuffle(self.carte) #mischia le carte
    
    #funzione che prende la prima carta del mazzo
    def pesca(self):
        return self.carte.pop(0)
    
    #funzione che crea tutte le carte
    def creaCarte(self):
        temp = []
        resto = 0
        for i in range (1,41):
            resto = i%10 #calcolo il modulo dell'elemento iesimo con 10 così da capire il valore della carta
            temp.append(Carta(Carta.semi[i//11], resto if resto != 0 else 10)) #aggiunge le carte al mazzo creandole con rispettivo seme (divisione intera per 11) e valore
        return temp #ritorna la lista di carte create

# Partita

In [558]:
class Partita():
    
    #costruttore
    def __init__(self):
        self.giocatore1 = Giocatore()
        self.giocatore2 = Giocatore()
        self.mazzo = Mazzo()
        self.briscola = "" #seme di briscola della partita
        self.first_throw = None #prima giocata di mano
        self.last_throw = None #seconda giocata di mano

    #funzione per controllare se la partita è finita
    def isFinita(self):
        if self.giocatore1.punteggio + self.giocatore2.punteggio == 120 and self.mazzo.carte == [] and self.giocatore1.mano == [] and self.giocatore2.mano == []: #controlla il punteggio totale dei giocatori e se le carte sono finite
            return True
        return False
    
    #funzione che definisce il primo turno
    def definisciTurno(self):
        temp = rand.randint(1,2) #randomicamente prende un intero scelto tra 1 e 2
        if temp == 1:
            self.giocatore1.turno = True 
        else:
            self.giocatore2.turno = True
    
    #funzione che distribuisce le carte a inizio partita
    def distribuisciCarte(self):
        first = self.whoHasTurn()
        second = None
        if first == self.giocatore1:
            second = self.giocatore2
        else:
            second = self.giocatore1
        for i in range (0,3): #tre carte ciascuno
            first.pesca(self.mazzo) #aggiunge la carta presa alla mano del giocatore
            second.pesca(self.mazzo)

    #funzione che definisce la briscola (dopo aver distribuito le carte a inizio partita, come in una partita reale)
    def definisciBriscola(self):
        ultima_carta = self.mazzo.carte.pop(0) #prende la prima carta del mazzo e questa sarà la briscola della partita
        self.mazzo.carte.append(ultima_carta) #viene inserita come ultima carta
        self.briscola = ultima_carta.seme #imposta il seme di briscola della partita
    
    #funzione che fa iniziare la partita
    def iniziaPartita(self):
        self.definisciTurno()
        self.distribuisciCarte()
        self.definisciBriscola()

    #funzione che setta la giocata del singolo giocatore ad ogni mano
    def setThrow(self, giocatore, carta, first):
        if first:
            self.first_throw = (carta, giocatore) #(la carta giocata, il giocatore che ha tirato per primo)
        else:
            self.last_throw = (carta, giocatore) #(la carta giocata, il giocatore che ha tirato per secondo)

    #funzione che resetta il singolo tiro ad ogni mano
    def resetThrow(self):
        self.first_throw = None
        self.last_throw = None

    #funzione che calcola il punteggio della singola mano
    def calcolaPunteggio(self):
        punti_a_terra = self.first_throw[0].punti
        seme_a_terra = self.first_throw[0].seme
        valore_a_terra = self.first_throw[0].valore

        last_carta_punti = self.last_throw[0].punti
        last_carta_seme = self.last_throw[0].seme
        last_carta_valore = self.last_throw[0].valore

        punti_mano = last_carta_punti + punti_a_terra

        if last_carta_seme == self.briscola and seme_a_terra == self.briscola:
            if last_carta_punti > punti_a_terra or last_carta_valore > valore_a_terra:
                self.last_throw[1].setPunteggio(punti_mano)
            elif last_carta_punti < punti_a_terra or last_carta_valore < valore_a_terra:
                self.first_throw[1].setPunteggio(punti_mano)
                self.cambiaTurno()
            return
        elif last_carta_seme == self.briscola:
            self.last_throw[1].setPunteggio(punti_mano)
            return
        elif seme_a_terra == self.briscola:
            self.first_throw[1].setPunteggio(punti_mano)
            self.cambiaTurno()
            return
        else:
            if last_carta_seme == seme_a_terra:
                if last_carta_punti > punti_a_terra or last_carta_valore > valore_a_terra:
                    self.last_throw[1].setPunteggio(punti_mano)
                elif last_carta_punti < punti_a_terra or last_carta_valore < valore_a_terra:
                    self.first_throw[1].setPunteggio(punti_mano)
                    self.cambiaTurno()
                return
            else:
                self.first_throw[1].setPunteggio(punti_mano)
                self.cambiaTurno()

    #funzione che cambia il turno ad ogni tiro
    def cambiaTurno(self):
        giocatore = None
        if self.giocatore1.turno == True:
            self.giocatore1.turno = False
            self.giocatore2.turno = True
            giocatore = self.giocatore2
        else:
            self.giocatore1.turno = True
            self.giocatore2.turno = False
            giocatore = self.giocatore1
        return giocatore

    #funzione per capire chi ha il turno ad ogni tiro
    def whoHasTurn(self):
        giocatore = None
        if self.giocatore1.turno == True:
            giocatore = self.giocatore1
        else:
            giocatore = self.giocatore2
        return giocatore

    def resetMano(self):
        first = self.whoHasTurn()
        second = None
        if first == self.giocatore1:
            second = self.giocatore2
        else:
            second = self.giocatore1
        
        if len(self.mazzo.carte) >= 2:
            first.pesca(self.mazzo)
            second.pesca(self.mazzo)
        self.resetThrow()
    
    def simulaPartita(self):
        self.iniziaPartita()
        while len(self.mazzo.carte) != 0:
            turno = self.whoHasTurn()
            turno.mossaRandom(self)
            turno = self.cambiaTurno()
            turno.mossaRandom(self)
            self.calcolaPunteggio()
            self.resetMano()

        for i in range (0,3):
            turno = self.whoHasTurn()
            turno.mossaRandom(self)
            turno = self.cambiaTurno()
            turno.mossaRandom(self)
            self.calcolaPunteggio()
            self.resetMano()
        
        print(self.giocatore1.punteggio)
        print(self.giocatore2.punteggio)




# Test

In [564]:
partita = Partita()
partita.simulaPartita()

34
86
